EXPLORING BOOK RATING DATA

In [13]:
liked_books = ["17381863", "35005834", "6514664", "7327624", "331839", "23167716", "2685097"]

In [14]:
!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [15]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [16]:
csv_book_mapping['0']

'34684622'

In [17]:
!wc -l goodreads_interactions.csv

228648343 goodreads_interactions.csv


In [18]:
!ls -lh | grep goodreads_interactions.csv

-rw-r--r-- 1 ashis 197609 4.1G Sep 17 23:00 goodreads_interactions.csv


FINDIND USERS WHO LIKE THE SAME BOOKS AS US

In [19]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

FINDING tHE BOOKS THOSE USERS LIKED

In [20]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [31]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [32]:
recs

,user_id,book_id,rating
0,1177,256683,5
1,1177,3777732,5
2,1177,1582996,5
3,1177,6752378,4
4,1177,7171637,5
...,...,...,...
405242,854754,6148028,5
405243,854754,9534,4
405244,854754,7260188,4
405245,854754,7108001,4


In [33]:
top_recs = recs["book_id"].value_counts().head(10)

In [34]:
top_recs = top_recs.index.values

In [37]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [38]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,theunschooledwizardsunwolfandstarhawk12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,bestfriendsforever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,theaeneidforboysandgirls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,allsfairyinloveandwaravalonwebofmagic8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,thedevilsnotebook


CREATING INITIAL BOOK RECOMMENDATION

In [50]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
73489,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,mockingjaythehungergames3
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,thehungergamesthehungergames1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilighttwilight1
569831,5,Harry Potter and the Prisoner of Azkaban (Harr...,1876252,https://www.goodreads.com/book/show/5.Harry_Po...,https://images.gr-assets.com/books/1499277281m...,harrypotterandtheprisonerofazkabanharrypotter3
833311,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catchingfirethehungergames2
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harrypotterandthesorcerersstoneharrypotter1
902688,136251,Harry Potter and the Deathly Hallows (Harry Po...,1784684,https://www.goodreads.com/book/show/136251.Har...,https://images.gr-assets.com/books/1474171184m...,harrypotterandthedeathlyhallowsharrypotter7
995137,15881,Harry Potter and the Chamber of Secrets (Harry...,1821802,https://www.goodreads.com/book/show/15881.Harr...,https://images.gr-assets.com/books/1474169725m...,harrypotterandthechamberofsecretsharrypotter2
1095301,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,thefaultinourstars
1248586,6,Harry Potter and the Goblet of Fire (Harry Pot...,1792561,https://www.goodreads.com/book/show/6.Harry_Po...,https://images.gr-assets.com/books/1361482611m...,harrypotterandthegobletoffireharrypotter4


In [53]:
recs

,user_id,book_id,rating
0,1177,256683,5
1,1177,3777732,5
2,1177,1582996,5
3,1177,6752378,4
4,1177,7171637,5
...,...,...,...
405242,854754,6148028,5
405243,854754,9534,4
405244,854754,7260188,4
405245,854754,7108001,4


IMPROVING THE RECOMMENDATION 

In [82]:
all_recs = recs["book_id"].value_counts()

In [83]:
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]

In [84]:
all_recs.head(5)

,book_id,book_count
0,2767052,109
1,3,99
2,41865,99
3,7260188,93
4,6148028,91


In [85]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [86]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])

In [87]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
60,35005834,50,DEVIL,334,https://www.goodreads.com/book/show/35005834-d...,https://images.gr-assets.com/books/1493402933m...,devil,7.485030
70,6514664,48,War,333,https://www.goodreads.com/book/show/6514664-war,https://images.gr-assets.com/books/1314464534m...,war,6.918919
80,2334506,46,Earth,538,https://www.goodreads.com/book/show/2334506.Earth,https://images.gr-assets.com/books/1314463746m...,earth,3.933086
85,17381863,45,Friends,536,https://www.goodreads.com/book/show/17381863-f...,https://images.gr-assets.com/books/1417983452m...,friends,3.777985
96,3118500,43,Wasteland,492,https://www.goodreads.com/book/show/3118500-wa...,https://images.gr-assets.com/books/1314464393m...,wasteland,3.758130
79,2334491,46,Hell,599,https://www.goodreads.com/book/show/2334491.Hell,https://images.gr-assets.com/books/1314464028m...,hell,3.532554
314,23167716,30,In,276,https://www.goodreads.com/book/show/23167716-in,https://images.gr-assets.com/books/1413517029m...,in,3.260870
4852,35378505,10,"Bane (Sinners of Saint, #4)",34,https://www.goodreads.com/book/show/35378505-bane,https://s.gr-assets.com/assets/nophoto/book/11...,banesinnersofsaint4,2.941176
664,6555025,23,The Takeover (Podiobook),205,https://www.goodreads.com/book/show/6555025-th...,https://s.gr-assets.com/assets/nophoto/book/11...,thetakeoverpodiobook,2.580488
1181,7327624,19,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,theunschooledwizardsunwolfandstarhawk12,2.578571


In [91]:
# all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False).head(10)

In [92]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [93]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
